In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import sys
print(sys.executable)
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path, models_path
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import matplotlib.pyplot as plt


/home/nwertheim/miniconda3/bin/python


In [3]:
data_file = os.path.join(univariate_data_path, 'merged_univariate.npy')
data = np.load(data_file, allow_pickle=True)
print(data.shape)

# Check for NaN or Inf values in signals
for sample in data:
    if np.isnan(sample['signal']).any() or np.isinf(sample['signal']).any():
        print(f"NaN or Inf detected in {sample['record_name']}")


print(data[0]['signal'].shape)
print(data[0])

(666,)
(7600, 1)
{'record_name': 'ice001_l_1of1', 'signal': array([[-1.7358303 ],
       [-0.30347557],
       [-0.40749874],
       ...,
       [-3.09738299],
       [-2.90981482],
       [-3.22768386]]), 'metadata': {'fs': 20, 'sig_len': 100000, 'n_sig': 16, 'base_date': None, 'base_time': None, 'units': ['mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV', 'mV'], 'comments': ['Info:', 'ID:ice001', 'Record type:labour', 'Record number:1/1', 'Age(years):31', 'BMI before pregnancy:23.3', 'BMI at recording:27.6', 'Gravidity:3', 'Parity:2', 'Previous caesarean:No', 'Placental position:Fundus', 'Gestational age at recording(w/d):39/3', 'Gestational age at delivery:39/3', 'Mode of delivery:Vaginal', 'Synthetic oxytocin use in labour:No', 'Epidural during labour:No', 'Comments for recording:', 'Electrodes placed 5-10 mins prior to beginning of recording.', 'Baby born 20 minutes after the end of the recording.']}}


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 275 has 2 dimension(s)

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split

# Create windows from the data
def create_windows(sequence, window_size, step_size):
    """ Create sliding windows from the sequence with specified window size and step size. """
    # Flatten the sequence if it's 2D (single channel)
    sequence = np.squeeze(sequence)  # Converts (7600, 1) -> (7600,)
    
    if sequence.ndim != 1:
        raise ValueError(f"Expected 1D array, got shape {sequence.shape}")
    
    return np.array([sequence[i:i+window_size] for i in range(0, len(sequence) - window_size + 1, step_size)])

# Masking function using patch-based masking
def mask_data(x, mask_ratio=0.5, patch_size=8):
    """ Apply patch-based masking to 1D signals """
    x_masked = np.copy(x)
    num_patches = x.shape[1] // patch_size  # Number of patches
    mask = np.random.rand(num_patches) < mask_ratio  # Randomly mask patches
    for i in range(num_patches):
        if mask[i]:
            x_masked[:, i * patch_size:(i + 1) * patch_size, :] = 0  # Zero out patches
    return x_masked

# Function to preprocess data: windowing, patch-based masking, etc.
def preprocess_data(data, window_size, step_size, patch_size, mask_ratio):
    # Create windows (assuming data is already in dictionary format)
    all_windows = [create_windows(record['signal'], window_size, step_size) for record in data]
    all_windows = np.concatenate(all_windows, axis=0)
    
    # Apply patch-based masking
    all_windows_masked = mask_data(all_windows, mask_ratio=mask_ratio, patch_size=patch_size)
    
    return all_windows, all_windows_masked

# Assuming `data` is a list of dictionaries
window_size = 12000  # 10-minute time series
step_size = 6000     # 50% overlap
patch_size = 24      # 24 samples per patch
mask_ratio = 0.5     # 50% patches are masked

# Preprocess data
all_windows, all_windows_masked = preprocess_data(data, window_size, step_size, patch_size, mask_ratio)

# Split data into train, validation, and test (80-10-10 split)
X_train, X_temp, y_train, y_temp = train_test_split(all_windows_masked, all_windows, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Check shapes for confirmation
print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")
print(f"Test set shape: {X_test.shape}")


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [5]:
from skopt import gp_minimize
from skopt.space import Integer, Real, Categorical
from sklearn.model_selection import train_test_split
import torch
import torch.optim as optim
import torch.nn.functional as F

# Define the search space
search_space = [
    Real(1e-5, 1e-2, "log-uniform", name='learning_rate'),
    Categorical([16, 32, 64], name='batch_size'),
    Integer(2, 5, name='num_layers'),
    Categorical([16, 32, 64], name='base_filters')  
]

# Create a function to build and train the model with hyperparameters
def build_model(learning_rate, num_layers, base_filters):
    input_shape = (500, 1)  # Shape of each time series window
    filters = [base_filters * (2**i) for i in range(num_layers)]  # Encoder: 2x growth
    filters += filters[-2::-1]  # Decoder: Mirror decreasing pattern
    
    # Ensure the number of input channels for the first layer is correct
    layers = []
    
    # Encoder layers
    in_channels = 1  # Input has 1 channel
    for i in range(num_layers):
        layers.append(nn.Conv1d(in_channels, filters[i], kernel_size=3, stride=1, padding=1))
        layers.append(nn.ReLU())
        in_channels = filters[i]  # Update input channels for next layer
    
    # Decoder layers (reverse the filters)
    for i in range(num_layers - 2, -1, -1):  # Skip the last filter in the encoder part
        layers.append(nn.ConvTranspose1d(in_channels, filters[i], kernel_size=3, stride=1, padding=1))
        layers.append(nn.ReLU())
        in_channels = filters[i]
    
    # Final layer (reconstruct to the original input size)
    layers.append(nn.Conv1d(in_channels, 1, kernel_size=3, stride=1, padding=1))
    
    # Combine all layers into a single sequential model
    model = nn.Sequential(*layers)
    return model


def objective(params):
    learning_rate, batch_size, num_layers, base_filters = params  # ✅ base_filters instead of filters_per_layer
    
    # Compute filter sizes
    filters = [base_filters * (2**i) for i in range(num_layers)]  # Encoder: 2x growth
    filters += filters[-2::-1]  # Decoder: Mirror decreasing pattern
    
    # Print the hyperparameters and filters for each layer
    print(f"Learning Rate: {learning_rate}, Batch Size: {batch_size}, Num Layers: {num_layers}, Base Filters: {base_filters}")
    print(f"Filters per layer: {filters}")  # Print filter sizes for each layer
    
    # Build the model
    model = build_model(learning_rate, num_layers, base_filters)
    
    # Split the data into training and validation sets
    X_train, X_val = train_test_split(all_windows, test_size=0.2, random_state=42)
    
    # Apply masking to the data
    X_train_masked = np.array([mask_data(window, mask_ratio=0.5) for window in X_train])
    X_val_masked = np.array([mask_data(window, mask_ratio=0.5) for window in X_val])
    
    # Convert to PyTorch tensors
    tensor_X_train = torch.tensor(X_train_masked, dtype=torch.float32).permute(0, 2, 1)  # [batch, channels, seq_len]
    tensor_X_val = torch.tensor(X_val_masked, dtype=torch.float32).permute(0, 2, 1)
    tensor_Y_train = torch.tensor(X_train, dtype=torch.float32).permute(0, 2, 1)
    tensor_Y_val = torch.tensor(X_val, dtype=torch.float32).permute(0, 2, 1)

    # Create datasets & loaders
    train_dataset = torch.utils.data.TensorDataset(tensor_X_train, tensor_Y_train)
    val_dataset = torch.utils.data.TensorDataset(tensor_X_val, tensor_Y_val)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    # Set up optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    model.train()
    for epoch in range(5):  # 5 epochs for tuning
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = F.mse_loss(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
    
    # Validation loss
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = F.mse_loss(outputs, targets)
            val_loss += loss.item()
    
    # Print the validation loss to the console
    print(f"Validation Loss: {val_loss / len(val_loader)}")
    
    # Append the results to a file
    with open("FCMAE_tuning.txt", "a") as file:
        file.write(f"Learning Rate: {learning_rate}, Batch Size: {batch_size}, Num Layers: {num_layers}, Base Filters: {base_filters}\n")
        file.write(f"Filters per layer: {filters}\n")  # Log filters for each layer
        file.write(f"Validation Loss: {val_loss / len(val_loader)}\n\n")
    
    return val_loss / len(val_loader)

# Run Bayesian Optimization
result = gp_minimize(objective, search_space, n_calls=15, random_state=42)

# Print the best hyperparameters and validation loss to the console
best_params = result.x
print(f"Best Hyperparameters: {best_params}")

with open("FCMAE_tuning.txt", "a") as file:
    file.write(f"Best Hyperparamters: {best_params}\n")


Learning Rate: 0.002452612631133679, Batch Size: 16, Num Layers: 4, Base Filters: 32
Filters per layer: [32, 64, 128, 256, 128, 64, 32]
